In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('D:\Manage\Project\spam.csv',encoding='latin1')


In [3]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [5]:
data.duplicated().sum()

403

In [6]:
data.drop_duplicates(keep='first',inplace=True)

In [7]:
data.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [8]:
data.shape

(5169, 2)

In [9]:
data.rename(columns={'v1':'Target','v2':'Text'},inplace=True)

In [10]:
data.sample()

,Target,Text
2661,ham,Do you know when dad will be back?


In [11]:
data['Target']=data['Target'].apply(lambda x: 1 if x== 'spam' else 0)

In [12]:
data.head()

,Target,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [44]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
def preprocess_text(text):
    ps = PorterStemmer()
    text = text.lower()  # تحويل إلى حروف صغيرة
    text = nltk.word_tokenize(text)  # تقسيم إلى كلمات
    text = [word for word in text if word.isalnum()]  # إزالة الرموز
    text = [word for word in text if word not in stopwords.words('english')]  # إزالة الكلمات الشائعة
    text = [ps.stem(word) for word in text]  # تقليل الكلمات إلى الجذر
    return " ".join(text)



In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Target'], test_size=0.2, random_state=42)


In [50]:
# تطبيق على بيانات التدريب والاختبار
X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# تحويل النصوص باستخدام TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# تدريب النموذج
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# تقييم النموذج
y_pred = model.predict(X_test_vectorized)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.965183752417795
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       889
           1       0.99      0.76      0.86       145

    accuracy                           0.97      1034
   macro avg       0.98      0.88      0.92      1034
weighted avg       0.97      0.97      0.96      1034



In [54]:
import pickle

# حفظ الموديل
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# حفظ TF-IDF Vectorizer
with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
